<a href="https://colab.research.google.com/github/usma11dia0/training_of_kaggle/blob/main/google_colab/titanic/titanic_youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot
import seaborn as sns

from pandas import Series, DataFrame
from collections import Counter

# トレーニングデータの準備
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# k-最近傍法
from sklearn.neighbors import KNeighborsClassifier

# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# データ読み込み
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/titanic_machine-learning-from-disaster/data/train.csv')
test_df =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/titanic_machine-learning-from-disaster/data/test.csv')
gender_submission_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/titanic_machine-learning-from-disaster/data/gender_submission.csv')

**全体の流れ**
1.   **モデルを作ってベースラインをとる**
*   数値データのみ
*   Nullなしデータのみ
*   k-最近傍法を使う  
2.  **有用な特徴量を探す**
*   カテゴリ特徴量をワンホットエンコーディング
*   スケール変換
*   Nullを埋める  

3.  **他のモデルを試してみて制度を確認する**
*   ロジスティック回帰
*   サポートベクタマシン
*   決定木
*   ランダムフォレスト
*   勾配ブースティング回帰木








1.**モデルを作ってベースラインをとる**

In [ ]:
# 数値データの確認(訓練データ)
train_df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [ ]:
# 数値データの確認(テストデータ)
test_df.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [ ]:
# nullの確認(訓練データ)
train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
# nullの確認(テストデータ)
test_df.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [ ]:
# 使えるデータ(数値型かつNullではない)で予測してみる。
# ※passenger_idなどのid系も不要
X = train_df[['Pclass','SibSp','Parch']]

# データフレーム型は2次元配列扱い。[]一つでSeries型にしておく。
y = train_df['Survived']

In [ ]:
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [ ]:
#　訓練データ、テストデータの生成
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
# k-最近傍法で予測
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [ ]:
knn.score(X_test, y_test)

0.6502242152466368

In [ ]:
X_for_submit = test_df[['Pclass','SibSp','Parch']]
submit = test_df[['PassengerId']]
submit['Survived'] = knn.predict(X_for_submit)

submit

<ipython-input-54-6e6f7e7a516f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit['Survived'] = knn.predict(X_for_submit)


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,0
415,1307,0
416,1308,0


In [ ]:
#　提出用csvファイルを保存
submit.to_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/titanic_machine-learning-from-disaster/submission/submit01.csv', index=False)

**２．有用な特徴量を探す**

In [ ]:
# 'Sex'を追加
X = train_df[['Pclass','SibSp','Parch','Sex']]
y = train_df['Survived']

In [ ]:
# ワンホットエンコーディングを利用する。
# → 文字列データをカテゴリーデータの数値(1 or 0)として変換することで、分析出来るようにすること
X = pd.get_dummies(X, columns=['Pclass', 'Sex'])
X

,SibSp,Parch,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male
0,1,0,0,0,1,0,1
1,1,0,1,0,0,1,0
2,0,0,0,0,1,1,0
3,1,0,1,0,0,1,0
4,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...
886,0,0,0,1,0,0,1
887,0,0,1,0,0,1,0
888,1,2,0,0,1,1,0
889,0,0,1,0,0,0,1


In [ ]:
#　訓練データ、テストデータの生成
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# k-最近傍法で予測
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

knn.score(X_test, y_test)

0.7982062780269058

In [ ]:
# Fareも追加。
X = train_df[['Pclass','SibSp','Parch','Sex', 'Fare']]
y = train_df['Survived']

# ワンホットエンコーディング
X = pd.get_dummies(X, columns=['Pclass', 'Sex'])

#　訓練データ、テストデータの生成
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# k-最近傍法で予測
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

knn.score(X_test, y_test)

0.7802690582959642

Fareと他の項目の**絶対値に差があるため揃える**。

In [ ]:
# 標準化を行う。
# →正規分布に無理やり変換。項目間の極端な値の差を埋めるために利用
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# k-最近傍法で予測
knn = KNeighborsClassifier()
knn.fit(X_train_scaled, y_train)

knn.score(X_test_scaled, y_test)

0.8161434977578476

In [ ]:
X_for_submit

,Pclass,SibSp,Parch
0,3,0,0
1,3,1,0
2,2,0,0
3,3,0,0
4,3,1,1
...,...,...,...
413,3,0,0
414,1,0,0
415,3,0,0
416,3,0,0


In [ ]:
X_for_submit = test_df[['Pclass','SibSp','Parch','Sex','Fare']]
submit = test_df[['PassengerId']]

X_for_submit = pd.get_dummies(X_for_submit, columns=['Pclass', 'Sex'])

# 平均値でnull値を埋める
X_for_submit['Fare'] = X_for_submit['Fare'].fillna(X_for_submit['Fare'].mean())

scaler = StandardScaler()
scaler.fit(X_train)

X_for_submit_scaled = scaler.transform(X_for_submit)

submit['Survived'] = knn.predict(X_for_submit_scaled)
submit

<ipython-input-62-500f8b18340c>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit['Survived'] = knn.predict(X_for_submit_scaled)


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
#　提出用csvファイルを保存
submit.to_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/titanic_machine-learning-from-disaster/submission/submit02.csv', index=False)

3.  **他のモデルを試してみて制度を確認する**

In [ ]:
X = train_df[['Pclass','SibSp','Parch','Sex', 'Fare']]
y = train_df['Survived']

# ワンホットエンコーディング
X = pd.get_dummies(X, columns=['Pclass', 'Sex'])

#　訓練データ、テストデータの生成
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# 標準化を行う。
# →正規分布に無理やり変換。項目間の極端な値の差を埋めるために利用
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
## サポートベクターマシン
from sklearn.svm import LinearSVC

svc = LinearSVC()
svc.fit(X_train_scaled, y_train)

svc.score(X_test_scaled, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.7847533632286996

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train_scaled, y_train)

rf.score(X_test_scaled, y_test)

0.8340807174887892

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbrt = GradientBoostingClassifier()
gbrt.fit(X_train_scaled, y_train)

gbrt.score(X_test_scaled, y_test)

0.8026905829596412

In [ ]:
X_for_submit = test_df[['Pclass','SibSp','Parch','Sex', 'Fare']]
submit = test_df[['PassengerId']]

# ワンホットエンコーディング
X_for_submit = pd.get_dummies(X_for_submit, columns=['Pclass', 'Sex'])

# 平均値でnull値を埋める
X_for_submit['Fare'] = X_for_submit['Fare'].fillna(X_for_submit['Fare'].mean())

#　訓練データ、テストデータの生成
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# # 標準化を行う。
# # →正規分布に無理やり変換。項目間の極端な値の差を埋めるために利用
# scaler = StandardScaler()
# scaler.fit(X_train)

X_for_submit_scaled = scaler.transform(X_for_submit)

submit['Survived'] = rf.predict(X_for_submit_scaled)
submit

<ipython-input-68-01e1ba612008>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit['Survived'] = rf.predict(X_for_submit_scaled)


,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
#　提出用csvファイルを保存
submit.to_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/titanic_machine-learning-from-disaster/submission/submit03.csv', index=False)